In [ ]:
import pandas as pd
import random
from sklearn import preprocessing
import numbers

dataTarget = []

def getNormalizeData(DATA):
    maxArr = []
    lenData = len(DATA)
    
    if(lenData == 0):
        return DATA
    
    lenDataInner = len(DATA[0])
    
    for i in range(lenDataInner):
        maxArr.append(DATA[0][i])
        
    for i in range(lenData):
        for j in range(lenDataInner):
            if (maxArr[j] < DATA[i][j]):
                maxArr[j] = DATA[i][j]

    NEWDATA = []
        
    for i in range(lenData):
        temp = []
        for j in range(lenDataInner):
            temp.append(float(DATA[i][j]) / float(maxArr[j]))
        
        NEWDATA.append(temp)
    
    return NEWDATA

def preProcessData(DATA, lenDATA, isSplitDataTarget):
    global dataTarget
    dataTrain = []
    datasetColumn = lenDATA

    le = preprocessing.LabelEncoder()

    dataTrainX = []

    # Hardcode dataset target name
    # dataTrain['target_name'] = ['<=50K', '>50K']
    
    # Fit data TARGET & transform for labelling non-number
    le.fit(DATA[datasetColumn - 1])
    dataTrainY = le.transform(DATA[datasetColumn - 1])

    # init target & length data target
    dataTarget = []
    lenDataTarget = len(dataTrainY)
    
    # FILL DATA TARGET
    for i in range(lenDataTarget):
        dataTarget.append(dataTrainY[i])

    # Fill temporary data feed
    for i in range(datasetColumn - 1):
        if(isinstance(DATA[i][0], numbers.Number)):
            tempIntToString = []
            for j in range(len(DATA[i])):
                tempIntToString.append(str(DATA[i][j]))
            le.fit(tempIntToString)
            dataTrainX.append(le.transform(tempIntToString))
        else:
            le.fit(DATA[i])
            dataTrainX.append(le.transform(DATA[i]))

    # FILL DATA FEED
    lenDataSet = len(dataTrainX[0])
    lenAttr = len(dataTrainX)
    for i in range(lenDataSet):
        temp = []
        for j in range(lenAttr):
            temp.append(dataTrainX[j][i])
        dataTrain.append(temp)

    return getNormalizeData(dataTrain)
    

dataset = pd.read_csv("CencusIncome.data.txt", header=None, delimiter=",")
# datatest = pd.read_csv("CencusIncome.test.txt", header=None, delimiter=",")

dataTrain = preProcessData(dataset, 15, True)
# dataTest = preProcessData(datatest, 15, True)

# outputnya ga bisa di print
# print len(dataTrain)
print ("Success")

In [ ]:
import math

# Menghitung jarak data antara data 1 terhadap data 2
# Jarak yang digunakan adalah jarak euclidean

def hitungjarakantardata(data1,data2):
    jarakabs = 0
    for i in range(len(data1)):
        jarakabs += pow(data1[i]-data2[i],2)
    return jarakabs

# Matrix jarak antar semua data di dataset
def matrixjarak(dataset):
    global jarakdata
    jarakdata = [[0 for x in range(len(dataset))] for y in range(len(dataset))] 
    
    for i in range(len(dataset)):
        for j in range(i+1,len(dataset)):
            jarakdata[i][j] = hitungjarakantardata(dataset[i],dataset[j])

# Menghitung jarak antar cluster
def hitungjarakantarcluster(cluster1,cluster2):
    global jarakdata
    jarakterdekat = 999
    for i in range(len(cluster1)):
        for j in range(len(cluster2)):
            if(jarakdata[cluster1[i]][cluster2[j]] < jarakterdekat):
                jarakterdekat = jarakdata[cluster1[i]][cluster2[j]]
    return jarakterdekat

def hitungjarakantarcluster(cluster1,cluster2):
    jarakterdekat = 999
    jarakterjauh = -1
    jarakakhir = 0
    jarak_average_group = 0
    
    if method_cluster_distance == 'average':
        total_data_distance = 0
        for i in range(len(cluster1)):
            for j in range(len(cluster2)):
                total_data_distance = total_data_distance + jarakdata[cluster1[i]][cluster2[j]]
        jarak_average = total_data_distance / (len(cluster1) * len(cluster2))
    elif method_cluster_distance == 'average_group':
        # average for cluster1
        total_distance_cluster1 = []
        for j in range(len(dataset[cluster1[0]])):
            total_distance_cluster1.append(0)
                
        for i in range(len(cluster1)):
            for j in range(len(dataset[cluster1[i]])):
                total_distance_cluster1[j] = total_distance_cluster1[j] + dataset[cluster1[i]][j]
        
        # compute the average
        for j in range(len(dataset[cluster1[0]])):
            total_distance_cluster1[j] = total_distance_cluster1[j] / len(cluster1)
        
        # average for cluster2
        total_distance_cluster2 = []
        for j in range(len(dataset[cluster2[0]])):
            total_distance_cluster2.append(0)
                
        for i in range(len(cluster2)):
            for j in range(len(dataset[cluster2[i]])):
                total_distance_cluster2[j] = total_distance_cluster2[j] + dataset[cluster2[i]][j]
        
        # compute the average
        for j in range(len(dataset[cluster2[0]])):
            total_distance_cluster2[j] = total_distance_cluster2[j] / len(cluster2)
        
        jarak_average_group = jarakdata[total_distance_cluster1][total_distance_cluster2]
    else:
        for i in range(len(cluster1)):
            for j in range(len(cluster2)):
                if method_cluster_distance == 'single':
                    if(jarakdata[cluster1[i]][cluster2[j]] < jarakterdekat):
                        jarakterdekat = jarakdata[cluster1[i]][cluster2[j]]
                else:
                    if(jarakdata[cluster1[i]][cluster2[j]] > jarakterjauh):
                        jarakterjauh = jarakdata[cluster1[i]][cluster2[j]]

            
    if method_cluster_distance == 'single':
        jarakakhir = jarakterdekat
    elif method_cluster_distance == 'complete':
        jarakakhir = jarakterjauh
    elif method_cluster_distance == 'average':
        jarakahir = jarak_average
    else:
        jarakakhir = jarak_average_group

    return jarakakhir

# Mencari cluster yang memiliki jarak terdekat 
def cariclusterterdekat(alcluster):
    jarakterdekat = 999
    for i in range(len(alcluster)):
        for j in range(i+1,len(alcluster)):
            if(hitungjarakantarcluster(alcluster[i],alcluster[j]) < jarakterdekat):
                jarakterdekat = hitungjarakantarcluster(alcluster[i],alcluster[j])
                idx = []
                idx.append(i)
                idx.append(j)
    return idx

In [ ]:
print ('Metode penghitungan jarak terdekat 2 cluster [single/complete/average/average_group]:')
method_cluster_distance = input()

#dataset = dataTrain
listrand = random.sample(range(0,32561),1000)
target = []
dataset = []
for i in range(len(listrand)):
    dataset.append(dataTrain[listrand[i]])
    target.append(dataTarget[listrand[i]])
clusterawal = []

for i in range(len(dataset)):
    temp = []
    temp.append(i)
    clusterawal.append(temp)

In [ ]:
matrixjarak(dataset)

cluster = []
for i in range(len(clusterawal)):
    cluster.append(clusterawal[i])

#print(len(dataset)," Cluster : ",cluster)

# Algoritma Utama Agglomerative

for i in range(1,len(dataset)-1):
    arrindex = cariclusterterdekat(cluster)
    indeks1 = arrindex[0]
    indeks2 = arrindex[1]
    for j in range(len(cluster[indeks2])):
        cluster[indeks1].append(cluster[indeks2][j])
    del cluster[indeks2]
        
   # print(len(dataset)-i,"Cluster : ",cluster)

In [ ]:
#Evaluasi dengan kelas target sebenarnya

banyaksama1 = 0;
banyaksama2 = 0;
banyaksama3 = 0;
banyaksama4 = 0;

for i in range(len(cluster[0])):
    if(dataTarget[cluster[0][i]] == 0):
        banyaksama1 += 1;
    else:
        banyaksama2 += 1;

for i in range(len(cluster[1])):
    if(dataTarget[cluster[0][i]] == 1):
        banyaksama3 += 1;
    else:
        banyaksama4 += 1;

print("Persentase kebenaran cluster jika dibandingkan dengan kelas sesungguhnya adalah : ")
print(100 * max(banyaksama1+banyaksama3,banyaksama2+banyaksama4) / (len(cluster[0]) + len(cluster[1]))," %")